In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

import cufflinks as cf
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

cf.go_offline()

%matplotlib inline
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error

from neuralforecast.auto import AutoInformer
from neuralforecast.tsdataset import TimeSeriesDataset
from neuralforecast.core import NeuralForecast

from datetime import datetime, timedelta
import plotly.graph_objects as go
import matplotlib.colors as mcolors
from neuralforecast.losses.pytorch import MQLoss
from tqdm import tqdm

In [2]:
# Load and process the MEASLES_ARIZONA data
df = pd.read_csv("outbreaks_disease_location.csv")
value_columns = [str(i) for i in range(60)]
series_values = df[value_columns].fillna(0).astype(float)
start_dates = pd.to_datetime(df["start_date"])

# Shuffle and split
shuffled_indices = df.sample(frac=1, random_state=42).index
split_point = int(0.8 * len(df))
train_indices = shuffled_indices[:split_point]
test_indices = shuffled_indices[split_point:]

In [3]:
train_records = []
for i, row in series_values.iloc[train_indices].iterrows():
    dates = pd.date_range(start="2000-01-01", periods=60, freq="W-SAT")
    for t, value in enumerate(row):
        train_records.append({"unique_id": f"Y{i+1}", "ds": dates[t], "y": value})
df_train = pd.DataFrame(train_records)

test_start_dates = start_dates.loc[test_indices] - pd.Timedelta(weeks=4)
df_test_all = []

for idx in test_indices:
    start_date = test_start_dates.loc[idx]
    row = series_values.loc[idx]
    dates = pd.date_range(start=start_date, periods=60, freq="W-SAT")
    for t, value in enumerate(row):
        df_test_all.append({"unique_id": f"Y_{idx}", "ds": dates[t], "y": value})

df_test_all = pd.DataFrame(df_test_all)

In [4]:
df_train["date"] = pd.to_datetime(df_train["ds"])
df_train.set_index("date", inplace = True)

df_test_all["date"] = pd.to_datetime(df_test_all["ds"])
df_test_all.set_index("date", inplace = True)

In [5]:
df_test_all

,unique_id,ds,y
date,,,
2022-08-13,Y_5607,2022-08-13,20775.0
2022-08-20,Y_5607,2022-08-20,16570.0
2022-08-27,Y_5607,2022-08-27,13995.0
2022-09-03,Y_5607,2022-09-03,12145.0
2022-09-10,Y_5607,2022-09-10,10686.0
...,...,...,...
1939-05-20,Y_7270,1939-05-20,0.0
1939-05-27,Y_7270,1939-05-27,0.0
1939-06-03,Y_7270,1939-06-03,0.0


In [6]:
class FixedModelInformerProcessor:
    def __init__(self, dates=[]):
        self.dates  = dates
        self.forecasts, self.eval_pairs = [], []
        self.testset = None
        self.nf, self.model = None, None
        self.selected_input_size = None
        self.maes, self.mses, self.mapes, self.nmses = [], [], [], []
        self.metrics_df  = pd.DataFrame(columns=["Unique_id","Reference Date","MAE","MSE","MAPE","NMSE"])
        self.display_df  = pd.DataFrame(columns=["Unique_id","Reference Date","Target End Date","GT","Quantile","Prediction"])
        self.wis_df      = pd.DataFrame()

    # ──────────────────────────────────────────
    def create_fixed_model(self, df_train, df_test, h: int, freq: str, level=[]):
        min_len = df_train.groupby("unique_id").size().min()
        max_in  = max(8, min_len - h - 1)
        self.testset = df_test

        def cfg(t):
            return {
                "input_size"   : t.suggest_int("input_size", 8, max_in),
                "learning_rate": t.suggest_float("learning_rate", 1e-4, 1e-2, log=True),
                "batch_size"   : t.suggest_categorical("batch_size",[16,32,64]),
                "random_seed"  : t.suggest_int("random_seed",1,99999),
                "accelerator":"gpu","devices":1,"strategy":"auto"
            }

        model = AutoInformer(
            h=h, backend="optuna",
            loss=MQLoss(level=level) if level else None,
            config=cfg)

        self.nf = NeuralForecast(models=[model], freq=freq)
        self.nf.fit(df=df_train)
        self.model = self.nf.models[0].model
        self.selected_input_size = self.model.hparams["input_size"]

    # ──────────────────────────────────────────
    def sliding_batch_by_index(self, h: int = 4):
        uid_dates = {u: g["ds"].sort_values().to_numpy()
                     for u, g in self.testset.groupby("unique_id")}
        T, s = min(len(d) for d in uid_dates.values()), self.selected_input_size
        rows = []

        for t in tqdm(range(s, T - h + 1), desc="index t"):
            batch, refs = [], []
            for uid, dates in uid_dates.items():
                win, ref = dates[t-s:t], dates[t-1]
                df_u  = self.testset[self.testset["unique_id"] == uid]
                df_in = df_u[df_u["ds"].isin(win)].copy()
                if len(df_in) == s:
                    df_in["unique_id"] = uid
                    batch.append(df_in); refs.append((uid, ref))
            if not batch: continue

            fcst_batch = self.nf.predict(df=pd.concat(batch)).set_index("ds")
            for uid, ref in refs:
                f  = fcst_batch[fcst_batch["unique_id"] == uid].reset_index()
                gt = (self.testset[self.testset["unique_id"] == uid]
                      .set_index("ds").loc[f["ds"]].reset_index())
                merged = pd.merge(f, gt, on=["unique_id","ds"], how="inner")
                merged["Reference Date"] = pd.to_datetime(ref).strftime("%Y-%m-%d")
                rows.append(merged)

        return pd.concat(rows, ignore_index=True)

    # ──────────────────────────────────────────
    def create_display_df(self, merged_df, model_name="AutoInformer"):
        recs = []
        for _, g in tqdm(merged_df.groupby("unique_id"), desc="display"):
            uid = g["unique_id"].iloc[0]; ref = g["Reference Date"].iloc[0]
            for col in g.columns:
                if not col.startswith(model_name): continue
                if col.endswith("median") or col == model_name:
                    q = 0.5
                elif "-lo-" in col:
                    lvl = int(col.split("-")[-1]); q = (1-lvl/100)/2
                elif "-hi-" in col:
                    lvl = int(col.split("-")[-1]); q = 1-(1-lvl/100)/2
                else:
                    continue
                tmp = g[["ds", col, "y"]].rename(columns={"ds":"Target End Date",
                                                           col:"Prediction",
                                                           "y":"GT"})
                tmp["Unique_id"] = uid
                tmp["Reference Date"] = ref
                tmp["Quantile"] = q
                recs.append(tmp)
        self.display_df = (pd.concat(recs)
                           .loc[:,["Unique_id","Reference Date","Target End Date",
                                   "GT","Quantile","Prediction"]]
                           .sort_values(["Unique_id","Reference Date",
                                         "Target End Date","Quantile"])
                           .reset_index(drop=True))

    # ──────────────────────────────────────────
    def efficient_compute_wis(self):
        if self.display_df.empty:
            raise ValueError("Run create_display_df first.")
        rows, df = [], self.display_df.sort_values(
            ["Unique_id","Reference Date","Target End Date","Quantile"])
        for (uid, ref, tgt), grp in tqdm(df.groupby(
                ["Unique_id","Reference Date","Target End Date"]), desc="WIS"):
            gt = grp["GT"].iloc[0]
            preds = grp.set_index("Quantile")["Prediction"]
            if 0.5 not in preds.index: continue
            rows.append({"Unique_id":uid,"Reference Date":ref,
                         "Target End Date":tgt,"GT":gt,
                         "WIS": abs(preds[0.5]-gt)})   # AE shortcut
        self.wis_df = pd.DataFrame(rows)

    # ──────────────────────────────────────────
    def calculate_metrics(self):
        for fc, gt in self.eval_pairs:
            y_true, y_pred = gt.iloc[:,1], fc.iloc[:,0]
            self.maes.append(mean_absolute_error(y_true, y_pred))
            self.mses.append(mean_squared_error(y_true, y_pred))
            self.mapes.append(mean_absolute_percentage_error(y_true, y_pred))
            self.nmses.append(self.mses[-1]/np.var(y_true) if np.var(y_true)!=0 else np.nan)

    def create_metrics_df(self):
        for i, (fc, _) in enumerate(self.eval_pairs):
            uid = fc["unique_id"].iloc[0]
            self.metrics_df.loc[len(self.metrics_df)] = [
                uid, self.dates[i], self.maes[i],
                self.mses[i], self.mapes[i], self.nmses[i]
            ]

In [7]:
test_ids = [f"Y_{i}" for i in test_indices]

In [8]:
processor = FixedModelInformerProcessor()

In [9]:
processor.create_fixed_model(df_train, df_test_all, h=4, freq="W-SAT", level = [10,20,30,40,50,60,70,80, 85,90,95])

[I 2025-06-25 20:02:09,998] A new study created in memory with name: no-name-a9763b3c-71dc-4bd6-95c5-6ba061c4b5f1
[rank: 0] Seed set to 45472
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

  | Name          | Type          | Params | Mode 
--------------------------------------------------------
0 | loss          | MQLoss        | 23     | train
1 | padder_train  | ConstantPad1d | 0      | train
2 | scaler        | TemporalNorm  | 0      | train
3 | enc_embedding | DataEmbedding | 384    | train
4 | dec_embedding | DataEmbedding | 384    | train
5 | encoder       | TransEncoder  | 199 K  | train
6 | decoder       | TransDecoder  | 144 K  | train
--------------------------------------------------------
344 K     Trainable params
23        Non-trainable params
344 K     Total params
1.379     Total estimated model params size (MB)
73        Modules in train mode
0         Mod

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_steps=5000` reached.


In [10]:
results = []

In [11]:
processor.selected_input_size

47

In [12]:
all_results = processor.sliding_batch_by_index(h=4)

index t:   0%|          | 0/10 [00:00<?, ?it/s]GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting: |          | 0/? [00:00<?, ?it/s]

index t:  10%|█         | 1/10 [00:39<05:53, 39.31s/it]GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting: |          | 0/? [00:00<?, ?it/s]

index t:  20%|██        | 2/10 [01:19<05:16, 39.62s/it]GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


...

index t:  80%|████████  | 8/10 [05:05<01:15, 37.90s/it]GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting: |          | 0/? [00:00<?, ?it/s]

index t:  90%|█████████ | 9/10 [05:45<00:38, 38.45s/it]GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Predicting: |          | 0/? [00:00<?, ?it/s]

index t: 100%|██████████| 10/10 [06:24<00:00, 38.44s/it]


In [15]:
all_results.to_csv('Informer_concatenated.csv')

In [14]:
all_results

,ds,unique_id,AutoInformer-median,AutoInformer-lo-95,AutoInformer-lo-90,AutoInformer-lo-85,AutoInformer-lo-80,AutoInformer-lo-70,AutoInformer-lo-60,AutoInformer-lo-50,...,AutoInformer-hi-40,AutoInformer-hi-50,AutoInformer-hi-60,AutoInformer-hi-70,AutoInformer-hi-80,AutoInformer-hi-85,AutoInformer-hi-90,AutoInformer-hi-95,y,Reference Date
0,2011-08-13,Y_1,0.015376,-0.001824,-0.035931,-0.013496,-0.004984,-0.004900,-0.024860,-0.025577,...,0.107602,0.156529,0.205063,0.280297,0.396730,0.482157,0.652432,1.021995,0.0,2011-08-06
1,2011-08-20,Y_1,0.027523,-0.001817,-0.035896,-0.013296,-0.004972,-0.005016,-0.024122,-0.022180,...,0.128998,0.181481,0.237070,0.319666,0.452376,0.552812,0.749573,1.165865,0.0,2011-08-06
2,2011-08-27,Y_1,0.024368,-0.001829,-0.035853,-0.013369,-0.005010,-0.005050,-0.024401,-0.023226,...,0.123497,0.174997,0.228529,0.309537,0.436369,0.533015,0.722059,1.124593,0.0,2011-08-06
3,2011-09-03,Y_1,0.021350,-0.001898,-0.035869,-0.013420,-0.005050,-0.005025,-0.025144,-0.024485,...,0.118147,0.168592,0.220793,0.300413,0.425355,0.520188,0.707415,1.110836,0.0,2011-08-06
4,2022-04-09,Y_1000,2.566213,-0.004147,-0.025277,-0.001889,0.008043,0.005082,0.455956,0.970998,...,4.511469,5.296147,6.384812,7.996187,10.777691,13.179578,16.938427,23.012135,0.0,2022-04-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86395,1955-04-16,Y_9998,-0.058410,-0.002331,-0.037491,-0.014341,-0.005416,-0.003303,-0.030552,-0.047188,...,-0.028111,-0.005648,0.013299,0.043555,0.088576,0.124088,0.211643,0.461736,0.0,1955-03-19
86396,1957-03-02,Y_9999,0.902782,-0.002956,-0.030273,-0.008452,-0.000977,-0.005572,0.108268,0.290571,...,1.658970,1.973932,2.405590,3.042092,4.158516,5.164028,6.787675,9.575441,0.0,1957-02-23
86397,1957-03-09,Y_9999,0.700485,-0.002932,-0.031291,-0.009288,-0.002028,-0.006254,0.069206,0.211201,...,1.308322,1.565778,1.914949,2.430275,3.342985,4.166986,5.517650,7.882787,0.0,1957-02-23
86398,1957-03-16,Y_9999,0.699488,-0.002931,-0.031295,-0.009291,-0.002032,-0.006261,0.069055,0.210848,...,1.306585,1.563728,1.912438,2.427081,3.338411,4.161106,5.509694,7.871290,0.0,1957-02-23


In [16]:
df = all_results.rename(columns={
    "ds"        : "Target End Date",
    "unique_id" : "Unique_id",
    "y"         : "GT"
})

# long‐format all quantile columns (everything that starts with AutoNBEATS-)
quantile_cols = [c for c in df.columns
                 if c.startswith("AutoInformer-")]

df_long = df.melt(
    id_vars = ["Unique_id", "Reference Date", "Target End Date", "GT"],
    value_vars = quantile_cols,
    var_name = "QuantileLabel",
    value_name = "Prediction"
)

def to_quantile(label: str) -> float:
    if "median" in label: return 0.5
    if "-lo-" in label:
        lvl = int(label.split("-")[-1]); return (1 - lvl/100) / 2
    if "-hi-" in label:
        lvl = int(label.split("-")[-1]); return 1 - (1 - lvl/100) / 2
    raise ValueError(label)

df_long["Quantile"] = df_long["QuantileLabel"].apply(to_quantile)

rows = []

group_iter = df_long.groupby(["Unique_id", "Reference Date", "Target End Date"])
for (uid, ref_date, tgt_date), g in tqdm(group_iter,
                                         total=len(group_iter),
                                         desc="point-wise metrics"):

    gt = g["GT"].iloc[0]
    preds = g.set_index("Quantile")["Prediction"]

    if 0.5 not in preds:          # need the median
        continue

    # point errors
    y_pred = preds[0.5]
    mae  = abs(y_pred - gt)
    mse  = (y_pred - gt) ** 2
    mape = abs((y_pred - gt) / gt) if gt else np.nan
    nmse = np.nan                # single point ⇒ var = 0

    # WIS for this timestamp
    qs  = sorted(q for q in preds.index if q != 0.5)
    n   = len(qs) // 2
    ae  = mae
    int_scores = 0
    for i in range(n):
        lo_q, hi_q = qs[i], qs[-(i + 1)]
        lo, hi     = preds[lo_q], preds[hi_q]
        alpha      = hi_q - lo_q
        int_scores += (hi - lo) \
                    + (2/alpha) * max(lo - gt, 0) \
                    + (2/alpha) * max(gt - hi, 0)

    wis = (ae + int_scores) / (1 + n)

    rows.append({
        "Unique_id"      : uid,
        "Reference Date" : ref_date,
        "Target End Date": tgt_date,
        "GT"             : gt,
        "Prediction"     : y_pred,
        "MAE"            : mae,
        "MSE"            : mse,
        "MAPE"           : mape,
        "NMSE"           : nmse,
        "WIS"            : wis
    })

pointwise_df = pd.DataFrame(rows).sort_values(
    ["Unique_id", "Reference Date", "Target End Date"])

point-wise metrics: 100%|██████████| 86400/86400 [00:49<00:00, 1745.02it/s]


In [17]:
pointwise_df

,Unique_id,Reference Date,Target End Date,GT,Prediction,MAE,MSE,MAPE,NMSE,WIS
0,Y_1,2011-08-06,2011-08-13,0.0,0.015376,0.015376,0.000236,NaN,NaN,0.307701
1,Y_1,2011-08-06,2011-08-20,0.0,0.027523,0.027523,0.000758,NaN,NaN,0.369680
2,Y_1,2011-08-06,2011-08-27,0.0,0.024368,0.024368,0.000594,NaN,NaN,0.351209
3,Y_1,2011-08-06,2011-09-03,0.0,0.021350,0.021350,0.000456,NaN,NaN,0.339422
4,Y_1,2011-08-13,2011-08-20,0.0,-0.018378,0.018378,0.000338,NaN,NaN,0.207673
...,...,...,...,...,...,...,...,...,...,...
86395,Y_9999,1957-02-16,1957-03-16,0.0,0.704864,0.704864,0.496833,NaN,NaN,4.410281
86396,Y_9999,1957-02-23,1957-03-02,0.0,0.902782,0.902782,0.815015,NaN,NaN,5.498482
86397,Y_9999,1957-02-23,1957-03-09,0.0,0.700485,0.700485,0.490679,NaN,NaN,4.340743
86398,Y_9999,1957-02-23,1957-03-16,0.0,0.699488,0.699488,0.489284,NaN,NaN,4.334477


In [19]:
pointwise_df.to_csv('Informer_ALL_STATS.csv')